In [1]:
print("Combined Running Started...")
import pathlib
import My_Funcs as my
import pandas as pd
import mne
import yasa
import numpy as np
import matplotlib.pyplot as plt
import openpyxl
import os
import h5py

Combined Running Started...


C:\Users\zloschin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\antropy\fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


In [5]:
"""
Setup All File Paths
"""

# .EDF file path
data_dir = pathlib.Path("D:/APNEA_project_edfs/File_Sig_Reruns/")

# Local table for reading in night division data
div_data = pathlib.Path("./Complete_Table_Final.xlsx")

# Read in division data to DataFrame
DivDF = pd.read_excel(div_data,engine='openpyxl')

"""
Storage Directories for Spindle detection
"""
# Spindle Folder
pipdir_spindles = pathlib.Path("./Combined_Running/Spindles_Data/")
pipdir_spindles.mkdir(parents=True, exist_ok=True)

# Shared directory for official runs
dict_dir_spindles = pathlib.Path("./Combined_Running/Spindles_Data/Detected_Spindles/")
dict_dir_spindles.mkdir(parents=True, exist_ok=True)

noise_dir_spindles = pathlib.Path("./Combined_Running/Spindles_Data/Detected_Noise/")
noise_dir_spindles.mkdir(parents=True, exist_ok=True)

# Official error directory
error_dir_spindles = pathlib.Path("./Combined_Running/Spindles_Data/Error_Storage/")
error_dir_spindles.mkdir(parents=True, exist_ok=True)

"""
Storage Directories for Slow wave detection
"""
# Directory for official running
pipdir_sw = pathlib.Path("./Combined_Running/SW_Data/")
pipdir_sw.mkdir(parents=True, exist_ok=True)

# Shared directory for official runs
dict_dir_sw = pathlib.Path("./Combined_Running/SW_Data/Detected_SW/")
dict_dir_sw.mkdir(parents=True, exist_ok=True)

noise_dir_sw = pathlib.Path("./Combined_Running/SW_Data/Detected_Noise/")
noise_dir_sw.mkdir(parents=True, exist_ok=True)

# Official error directory
error_dir_sw = pathlib.Path("./Combined_Running/SW_Data/Error_Storage/")
error_dir_sw.mkdir(parents=True, exist_ok=True)

# Plots of average slow wave
plot_dir_sw = pathlib.Path("./Combined_Running/SW_Data/Plots/")
plot_dir_sw.mkdir(parents=True, exist_ok=True)

"""
Storage lists for participant items
"""
N2D = []
N3D = []
N23D = []

N2D_SW = []
N3D_SW = []
N23D_SW = []

IDs = []
AllIDs = []

EndTimeN1 = []
StartTimeN2 = []

In [6]:
"""
Setup file generator for the for loop
"""
files = pathlib.Path(data_dir).glob('*.edf')

print("Entering Participant Loop...")
for file in files:
    try:
        with open(file, 'rb') as f:
            # Get ID and Night Number from the File Name
            split_array = str(file).split('_')
            night = int(split_array[-1][0])
            ID =int(split_array[-2])
            IDs.append(ID)
            print(f"ID: {ID} Night: {night}")

            """
            Read in EDF
            """
            print(f"Reading in Patient {ID}")
            rawImport = mne.io.read_raw_edf(file, preload=True)
            eeg = rawImport["EEG"][0][0]

            """
            Creation of our data dictionary
            """
            print("Creating dict...")
            # Create epoch vector
            epochVec = my.create_epochs(rawImport["EEG"][0][0], rawImport.info["sfreq"], 30)

            # Create epoch time vector
            epochTimeVec = my.create_epochs(rawImport["EEG"][1], rawImport.info["sfreq"], 30)

            # Create Dictionary for organization
            data = my.create_dict(rawImport["EEG"][0][0], rawImport.info["sfreq"], rawImport["EEG"][1], epochVec,
                                epochTimeVec)
            
            """
            Artifact removal (create label vec with artifact locations as Nan)
            """
            print("Creating artifact label vector...")
            data["ArtZero"] = np.zeros(len(data["EEG"]))
            for i in range(len(data["ArtZero"])):
                if abs(data["EEG"][i]) >= 250:
                    data["ArtZero"][i] = np.nan

            """
            Band pass filter the EEG data (10-16 Hz)
            """
            print("Bandpass filtering...")
            data["EEG_Filt"] = my.butter_bandpass_filter(data["EEG"], 10, 16, data["fs"], order=5)

            """
            Upper Beta pass filter to graph (20-50 Hz)
            """
            print("Beta pass filtering...")
            data["EEG_Beta"] = my.butter_bandpass_filter(data["EEG"], 20, 50, data["fs"], order=5)
            data["Beta_Labels"] = np.zeros(len(data["EEG_Beta"]))

            # For any threshold event, change the beta label vec to 1
            for i in range(len(data["EEG_Beta"])):
                if data["EEG_Beta"][i] > 10:
                    data["Beta_Labels"][i - 64:i + 64] = 1

            """
            Remove artifacts using the label vector
            """
            print("Removing artifacts...")
            data["EEG_ArtZero"] = data["EEG"]
            for i in range(len(data["EEG"])):
                if np.isnan(data["ArtZero"][i]):
                    data["EEG_ArtZero"][i] = 0
                    data["EEG_Filt"][i] = 0
                    data["EEG_Beta"][i] = 0
            
            """
            Load hypnogram 
            """
            hypno_path = 'D:/APNEA_project_edfs/ReadyForChecking/'
            print(hypno_path+os.sep+'psgHypno-%s_%s.mat'% (ID, night))
            hypno = h5py.File(hypno_path+os.sep+'psgHypno-%s_%s.mat'% (ID, night)) 

            print("Len hypno")
            print(len(hypno['dat']))
            print("Len eeg")
            print(len(rawImport["EEG"][0][0]))

            hypno = hypno['dat']
            hypno = np.array(hypno)
            selectedHypno = hypno[:,1]

            # Create a dictionary to map values to strings
            mapping = {0: 'W', 1: 'N1', 2: 'N2', 3: 'N3', 5: 'R'}

            # Replace the values in selectedHypno with the corresponding strings
            # If a value is not in the mapping, replace it with 'W'
            hypno = np.array([mapping.get(x, 'W') for x in selectedHypno])

            """
            Yasa spindle detection and noise elimination
            """
            # Convert the hypnograms to numbers
            HypnoEnum = my.hypno_to_plot_art(hypno)

            # Upsample the Hypnogram to be the same length as the data array
            hypnoEnumUpsampled = my.upsample_hypno_to_data(HypnoEnum, data)
            
            print("Detecting Spindles...")
            # Detect spindles
            spindles = my.detect_spindles(data, hypnoEnumUpsampled, night=2, samples1=0)

            """
            11) Save spindle data to excel
            """
            print("Saving Spindle Data...")
            n1File = "Spindle_%s_" % ID
            n1File = n1File + "%s.xlsx" % night
            n1Path = dict_dir_spindles / n1File
            spindles.to_excel(n1Path)


            """
            12) Calculating NREM Spindle Densities
            """
            print("Calculating Spindle Density...")
            NREM2_Density, NREM3_Density, NREM23_Density = my.nrem_densities(spindles, HypnoEnum)

            N2D.append(NREM2_Density)
            N3D.append(NREM3_Density)
            N23D.append(NREM23_Density)
    except Exception as e:
        print("**********************************")
        print(e)
        error_file = '%s_%s.txt' % (ID, 'ErrorMSG')
        error_path = error_dir_spindles/ error_file
        with open(error_path, 'w') as file:
            file.write(str(e))
        continue

    try:
        print("Entering Slow Wave Block...")
        """
        Yasa Slow wave detection
        """
        # Upsample the Hypnogram to be the same length as the data array
        hypnoEnumUpsampled = my.upsample_hypno_to_data(HypnoEnum, data)
        
        print("Detecting Slow Waves...")
        SW_Summary, ave_sw = my.detect_slow_waves(data, hypnoEnumUpsampled)
        

        """
        Calculate slow wave densities
        """
        print("Calculating Slow Wave Densities...")
        NREM2_Density, NREM3_Density, NREM23_Density = my.nrem_densities(SW_Summary, HypnoEnum)

        N2D_SW.append(NREM2_Density)
        N3D_SW.append(NREM3_Density)
        N23D_SW.append(NREM23_Density)
        
        
        """
        11) Save Slow wave data to excel
        """
        print("Saving Slow Wave Data...")
        n1File = "Slow_Wave_%s_" % ID
        n1File = n1File + "%s.xlsx" % night
        n1Path = dict_dir_sw / n1File
        SW_Summary.to_excel(n1Path)
        
        
        """
        Graph the average slow wave and save it
        """
        print("Plotting Average Slow Wave and Saving...")
        plt.plot(ave_sw)
        plt.xlabel('Time (s)')
        plt.ylabel('Normalized Ampliteude (uV)')
        plt.title('Average SW Graph ')
        plt.grid(True)
        plt.savefig(f'./Combined_Running/SW_Data/Plots/Slow_Wave_Graph_{ID}_AmpFiltered.png')
        plt.clf()
        
    except Exception as e:
        print("##################################")
        error_file = '%s_%s.txt' % (ID, 'ErrorMSG')
        error_path = error_dir_sw / error_file
        with open(error_path, 'w') as file:
            file.write(str(e))
        continue


Entering Participant Loop...
ID: 451 Night: 2
Reading in Patient 451
Extracting EDF parameters from D:\APNEA_project_edfs\File_Sig_Reruns\Zach_451_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5045759  =      0.000 ... 19709.996 secs...


C:\Users\zloschin\AppData\Local\Temp\ipykernel_280\3236609621.py:21: RuntimeWarning: Physical range is not defined in following channels:
RTC
  rawImport = mne.io.read_raw_edf(file, preload=True)


Creating dict...
Creating artifact label vector...
Bandpass filtering...
Beta pass filtering...
Removing artifacts...
D:/APNEA_project_edfs/ReadyForChecking/\psgHypno-451_2.mat
Len hypno
657
Len eeg
5045760
Detecting Spindles...
Detecting spindles...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Creating spindle dataframe...
Removing everything but spindles...
151372800
Saving Spindle Data...
Calculating Spindle Density...
278
115
Entering Slow Wave Block...
Detecting Slow Waves...


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


Creating spindle dataframe...
Removing everything but spindles...
192844800
Saving Spindle Data...
Calculating Spindle Density...
360
268
Entering Slow Wave Block...
Detecting Slow Waves...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


Calculating average vectors...
Calculating Slow Wave Densities...
360
268
Saving Slow Wave Data...
Plotting Average Slow Wave and Saving...


<Figure size 640x480 with 0 Axes>

In [7]:
"""
Save the Spindle Density Tables
"""
print("Saving Spindle Density Table...")
densities = pd.DataFrame()

densities["N2"] = N2D
densities["N3"] = N3D
densities["N23"] = N23D

filename = 'Spindle_Densities.xlsx'
spindle_density_path  = pipdir_spindles / filename
densities.to_excel(spindle_density_path)


"""
Save the Slow Wave Density Tables
"""
print("Saving Slow Wave Density Table")
densities_sw = pd.DataFrame()
densities_sw["N2"] = N2D_SW
densities_sw["N3"] = N3D_SW
densities_sw["N23"] = N23D_SW

filename = 'SW_Densities.xlsx'
sw_density_path = pipdir_sw / filename
densities_sw.to_excel(sw_density_path)

Saving Spindle Density Table...
Saving Slow Wave Density Table
